Simple Iterative Methods
=

In this chapter we learn

* The Richardson Iteration
* The Gradient Method
* Using Preconditioners

We are given a linear system of equations

$$
A x = b.
$$

The matrix $A$ is so large such that direct elimination is not a good option. 
Although this section applies to linear systems in general, we think 
of equations arising from finite element discretization. Other numerical methods for partial differential equations lead to similar systems. Matrices from graph networks are another example.

The situation is advantageous if the matrix is
* symmetric:  $$A = A^T$$
* positive definite: $$x^T A x > 0 \qquad \forall \, 0 \neq x \in R^n$$

We call symmetric and positive definite matrices SPD.

We do not assume that the matrix is stored as a dense matrix, i.e. all $n^2$ entries are stored in a two dimensional array.

We only assume that the matrix-vector product

$$
y := A * x
$$

is computationally available. 

Typically, the algorithmic complexity of the matrix-vector product is $O(n)$. 

Often one stores the matrix in a sparse matrix format, such as the CSR (Compressed Sparse Row) format. https://de.wikipedia.org/wiki/Compressed_Row_Storage https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html


More general, the matrix-vector product can be any linear operator. We can think of the sum or product of linear operators, multiplying with the inverse of the diagonal, or solving a triangular (sparse) system by forward/backward substitution

The Richardson Iteration
---
also called simple iteration is the fixed point iteration

$$
x^{k+1} := x^k + \alpha \, (b - A x^k)
$$

with an arbitrary starting value $x^0 \in R^n$, and a properly cosen damping paramter $\alpha$.

The solution $x^\ast$ is a fixed point of the iteration (since $b - A x^\ast = 0$).

If we define the error as
$$
e^k = x^k - x^\ast,
$$

the error propagation from one step to the next is

\begin{eqnarray*}
e^{k+1} & = & x^{k+1} - x^\ast = x^k + \alpha (b - A x^k) - x^\ast \\
& = & x^k - x^\ast + \alpha A (x^\ast - x^k) = (I - \alpha A) (x^k - x^\ast)
\end{eqnarray*}

This means the new error is obtained from the old error by the error propagation matrix

$$
e^{k+1} = (I - \alpha A) \, e^k
$$

Two strategies to verify convergence are:

* prove that the convergence radius 

$$
\rho(I - \alpha A) := \max_{\lambda \in \sigma(I - \alpha A)} |\lambda| < 1
$$

* find some norm $\| \cdot \|$ such that the matrix norm (=operator norm)

$$
\| I - \alpha A \| := \sup_{x \in R^n} \frac{ \| (I - \alpha A) x \| }{ \| x \| } < 1
$$

The first one, $\rho < 1$, only provides asymptotic convergence. This is easily proven if A is diagonizable, i.e. it features a full set of eigenvectors $z^j$ and eigenvalues $\lambda_j$. Expand the initial error as
$$
e^0 = \sum_j e^0_j z^j,
$$
then
$$
e^k = \sum_j (1-\alpha \lambda_j)^k e^0_j z^j
$$
and
$$
\| e^k \| \leq \rho^k  \sum_j | e^0_j | \| z^j \|
$$
This means $\| e^k \| \leq C \rho^k$, the the error does not have to decrease monotonically.

However, if $\| I - \alpha A \| < 1$, then
$$
\| e^k \| \leq \| I - \alpha A \| \, \| e^k \|,
$$
and the error decreases in every interation step. Note that the matrix norm is the operator norm generated by the vector norm.

Some facts:
* If the norm $\| \cdot \|$ is generated by an inner product $\left< \cdot, \cdot \right>$ (parallelogram identity), and $M$ is some self adjoint matrix with respect to this inner product, i.e.
$$
\left< M x, y \right>  = \left< x, M y \right>,
$$
  then $\rho(M) = \| M \|$

  If $\left< \cdot , \cdot \right>$ is the Euklidean inner product, then $M$ is self-adjoint exactly when $M$ is symmetric.
  
* Every operator operator norm is bounded by the spectral radius. There exists some norm such that the operator norm is arbitrary close to the spectral radius, i.e.
$$
\rho(M) = \sup_{ \text{norms} \| \cdot \| } \| M \|
$$

Optimizing the relaxation parameter $\alpha$
---
Let $A$ be SPD, and let $\sigma(A) = \{ \lambda_i \in R \}$ with $0 < \lambda_1 \leq \lambda_2 \ldots \leq \lambda_n$.

Then the eigenvalues of $M = I - \alpha A$ are $\{ 1 - \alpha \lambda_i  \}$. 

Whenever we choose 
$$
0 < \alpha < \frac{2}{\lambda_n}
$$
we obtain $\rho(M) < 1$ and a convergent iteration.


The spectral radius of $M$ is 

$$
\rho(M) = \max \{ | 1 - \alpha \lambda_i| \}  = 
\max \{ 1 - \alpha \lambda_1, - (1-\alpha \lambda_n) \}
$$

The maximum is minimized if we choose $\alpha$ optimally such that
$$
1 - \alpha \lambda_1 = - (1 - \alpha \lambda_n),
$$
i.e.
$$
\alpha_{\text{opt}} = \frac{2}{\lambda_1 + \lambda_n} 
$$
leading to the optimal convergence rate

$$
\rho_{\text{opt}} = \frac{\lambda_n - \lambda_1}{\lambda_n+\lambda_1}
\approx 1 - 2 \frac{\lambda_1}{\lambda_n} = 1 - \frac{2}{\kappa(A)}
$$

with the condition number $\kappa(A) = \lambda_n(A) / \lambda_1(A)$.

Experiments with the Richardson iteration
---

In [37]:
from ngsolve import *
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [52]:
fes = H1(mesh, order=1)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx+10*u*v*dx).Assemble()
f = LinearForm(x*y*v*dx).Assemble()
gfu = GridFunction(fes)

we determine (an approximation to) the largest eigenvalue by a few steps of the power iteration:

In [53]:
hv = gfu.vec.CreateVector()
hv2 = gfu.vec.CreateVector()
hv.SetRandom()
hv.data /= Norm(hv)
for k in range(100):
    hv2.data = a.mat * hv
    rho = Norm(hv2)
    print (rho)
    hv.data = 1/rho * hv2

1.8323633200637919
4.342246530037763
4.638526197596456
4.821638717050277
4.967963746251901
5.091948210157183
5.197879048051337
5.287768590486316
5.3632246550955145
5.4259167246077435
5.4775992841685825
5.520006874119458
5.554747405232093
5.583235834991281
5.606670213611571
5.6260377244707
5.6421370682117775
5.6556070476156775
5.666955263050212
5.676583910698987
5.684811600703039
5.691891148425725
5.698023763285679
5.7033702148128285
5.70805955390402
5.712195899443136
5.715863713530388
5.71913190438916
5.72205702308904
5.7246857605230295
5.727056903737233
5.729202873859194
5.731150939482923
5.7329241776228255
5.734542237731486
5.7360219515837825
5.737377822126339
5.738622416960582
5.739766686424397
5.740820221849119
5.74179146618257
5.7426878865490405
5.743516116283874
5.744282072398003
5.744991053192397
5.745647819775292
5.746256664475504
5.746821468546988
5.747345751087219
5.74783271071761
5.748285261276698
5.748706062539826
5.749097546789473
5.7494619419087565
5.749801291548375
5.750

In [54]:
alpha = 1 / rho
res = f.vec.CreateVector()
gfu.vec[:] = 0
for k in range(1000):
    res.data = f.vec - a.mat * gfu.vec
    print ("iteration", k, "res=", Norm(res))
    gfu.vec.data += alpha * res

iteration 0 res= 0.02956042994115231
iteration 1 res= 0.02832352390846811
iteration 2 res= 0.027515656766641614
iteration 3 res= 0.02685512680509232
iteration 4 res= 0.026271916569890155
iteration 5 res= 0.025735919914898223
iteration 6 res= 0.02523206822374322
iteration 7 res= 0.02475214919785126
iteration 8 res= 0.02429135958160777
iteration 9 res= 0.023846698060094006
iteration 10 res= 0.02341616768256817
iteration 11 res= 0.02299835964366913
iteration 12 res= 0.022592226121503373
iteration 13 res= 0.022196951025922493
iteration 14 res= 0.02181187350224868
iteration 15 res= 0.021436440928753464
iteration 16 res= 0.021070178988086895
iteration 17 res= 0.02071267195631616
iteration 18 res= 0.020363549303067996
iteration 19 res= 0.02002247630893746
iteration 20 res= 0.01968914731389345
iteration 21 res= 0.019363280735496046
iteration 22 res= 0.019044615307707694
iteration 23 res= 0.01873290718121517
iteration 24 res= 0.0184279276449314
iteration 25 res= 0.018129461304268346
iteration 2

iteration 606 res= 1.0393992832155842e-05
iteration 607 res= 1.0264368433965542e-05
iteration 608 res= 1.0136360594773822e-05
iteration 609 res= 1.0009949154249758e-05
iteration 610 res= 9.88511420348901e-06
iteration 611 res= 9.761836081866145e-06
iteration 612 res= 9.640095373979087e-06
iteration 613 res= 9.519872906506788e-06
iteration 614 res= 9.401149745279463e-06
iteration 615 res= 9.283907192259181e-06
iteration 616 res= 9.168126782604539e-06
iteration 617 res= 9.05379028169915e-06
iteration 618 res= 8.940879682395496e-06
iteration 619 res= 8.829377202102057e-06
iteration 620 res= 8.719265279973858e-06
iteration 621 res= 8.610526574190036e-06
iteration 622 res= 8.503143959206994e-06
iteration 623 res= 8.397100523039065e-06
iteration 624 res= 8.292379564628237e-06
iteration 625 res= 8.188964591202277e-06
iteration 626 res= 8.086839315668222e-06
iteration 627 res= 7.985987654030177e-06
iteration 628 res= 7.886393722916307e-06
iteration 629 res= 7.788041837018554e-06
iteration 630 

In [55]:
from ngsolve.webgui import Draw
Draw (gfu)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2101-139-g7eb2ca6ee', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…